In [28]:
import requests
from dotenv import load_dotenv
import os
import pandas as pd
import steamspypi
import steamreviews


load_dotenv()
API_KEY = os.getenv("API_KEY")


# Collect the 100 most popular games on steam

In [8]:
data_request = {"request": "top100owned"}
data = steamspypi.download(data_request)
df = pd.DataFrame(data.values())
df = df[['name', 'appid']]
display(df.head())


,name,appid
0,Dota 2,570
1,Counter-Strike: Global Offensive,730
2,PUBG: BATTLEGROUNDS,578080
3,Palworld,1623730
4,New World,1063730


{'570': {'appid': 570, 'name': 'Dota 2', 'developer': 'Valve', 'publisher': 'Valve', 'score_rank': '', 'positive': 1826884, 'negative': 404152, 'userscore': 0, 'owners': '100,000,000 .. 200,000,000', 'average_forever': 45317, 'average_2weeks': 1643, 'median_forever': 1002, 'median_2weeks': 962, 'price': '0', 'initialprice': '0', 'discount': '0', 'ccu': 585441}, '730': {'appid': 730, 'name': 'Counter-Strike: Global Offensive', 'developer': 'Valve', 'publisher': 'Valve', 'score_rank': '', 'positive': 7010454, 'negative': 1025813, 'userscore': 0, 'owners': '100,000,000 .. 200,000,000', 'average_forever': 35101, 'average_2weeks': 728, 'median_forever': 6440, 'median_2weeks': 335, 'price': '0', 'initialprice': '0', 'discount': '0', 'ccu': 1107148}, '578080': {'appid': 578080, 'name': 'PUBG: BATTLEGROUNDS', 'developer': 'KRAFTON, Inc.', 'publisher': 'KRAFTON, Inc.', 'score_rank': '', 'positive': 1374069, 'negative': 979688, 'userscore': 0, 'owners': '50,000,000 .. 100,000,000', 'average_fore

In [9]:


# Define a function to get the tags for a game
def get_tags(appid):
    data_request = {"request": "appdetails", "appid": appid}
    data = steamspypi.download(data_request)
    return data['tags']

# Apply the function to the 'appid' column of the DataFrame
df['tags'] = df['appid'].apply(get_tags)

# Print the DataFrame
print(df)

                                name    appid  \
0                             Dota 2      570   
1   Counter-Strike: Global Offensive      730   
2                PUBG: BATTLEGROUNDS   578080   
3                           Palworld  1623730   
4                          New World  1063730   
..                               ...      ...   
95                   Dota Underlords  1046930   
96                   Project Zomboid   108600   
97    Counter-Strike: Condition Zero       80   
98                    Minion Masters   489520   
99                Sons Of The Forest  1326470   

                                                 tags  
0   {'Free to Play': 59545, 'MOBA': 19960, 'Multip...  
1   {'FPS': 90048, 'Shooter': 64762, 'Multiplayer'...  
2   {'Survival': 14603, 'Shooter': 12471, 'Battle ...  
3   {'Multiplayer': 988, 'Open World': 983, 'Creat...  
4   {'Open World': 771, 'Massively Multiplayer': 7...  
..                                                ...  
95  {'Free to Play'

# Construct player networks

Does it make sense to prune the friends of seed players based on wether they have the original game or not? The two options are smartly choosing the seed players or recursively looking at friends of friends who have the game until we have a big enough network.

In [46]:
def get_friends_with_game(player_id, app_id):
    """ 
    Given a player_id and an app_id, returns a list of the player's friends who own the game.
    """
    all_friends = []
    url = f'http://api.steampowered.com/ISteamUser/GetFriendList/v0001/?key={API_KEY}&steamid={player_id}&relationship=friend'
    response = requests.get(url)
    friend_list = response.json()
    try:
        for item in friend_list["friendslist"]["friends"]:
            all_friends.append(item["steamid"])
    except:
        print("Error getting friends.")
    friends_with_game = []
    for friend in all_friends:
        games = []
        url = f'http://api.steampowered.com/IPlayerService/GetOwnedGames/v0001/?key={API_KEY}&steamid={friend}&format=json'
        response = requests.get(url)
        game_list = response.json()
        try:
            for item in game_list["response"]["games"]:
                games.append(item["appid"])
        except:
            continue
        if app_id in games:
            friends_with_game.append(friend)
    return friends_with_game
    
def get_friends(player_id):
    """ 
    Given a player_id, returns a list of the player's friends.
    """
    all_friends = []
    url = f'http://api.steampowered.com/ISteamUser/GetFriendList/v0001/?key={API_KEY}&steamid={player_id}&relationship=friend'
    response = requests.get(url)
    friend_list = response.json()
    try:
        for item in friend_list["friendslist"]["friends"]:
            all_friends.append(item["steamid"])
    except:
        print("Error getting friends.")
    return all_friends    
    
def get_ownership_network(app_id, params, seed_size=10, min_net_size=100):
    """ 
    Given an app_id, returns a list of players who own the game and their friends who own the game.
    """
    # Get seed players
    review_dict, query_count = steamreviews.download_reviews_for_app_id(app_id, chosen_request_params=params)
    players = []
    for key in review_dict["reviews"].keys():
        if len(players) < seed_size:
            players.append(review_dict["reviews"][key]["author"]["steamid"])
    nodes = players
    while len(nodes) < min_net_size:
        new_nodes = []
        for player in players:
            friends = get_friends_with_game(player, app_id)
            
            for friend in friends:
                if friend not in nodes:
                    new_nodes.append(friend)
                    print(f"Added {friend} to the network.")
                    if len(nodes) + len(new_nodes) >= min_net_size:
                        nodes += new_nodes
                        return nodes
        nodes += new_nodes
        players = new_nodes
    return nodes

def get_non_ownership_network(app_id, params, seed_size=10, min_net_size=100):
    """ 
    Given an app_id, returns a friendship network of seed players. Friends of seeded players don't necessarily own the game.
    """
     # Get seed players
    review_dict, query_count = steamreviews.download_reviews_for_app_id(app_id, chosen_request_params=params)
    players = []
    for key in review_dict["reviews"].keys():
        if len(players) < seed_size:
            players.append(review_dict["reviews"][key]["author"]["steamid"])
            
    network = players
    while len(network) < min_net_size:
        new_nodes = []
        for player in players:
            friends = get_friends(player)
            for friend in friends:
                if friend not in network:
                    new_nodes.append(friend)
                    print(f"Added {friend} to the network.")
                    if len(network) + len(new_nodes) >= min_net_size:
                        network += new_nodes
                        return network
        
        

In [49]:
app_id = 730

request_params = dict()
request_params['language'] = 'english'
request_params['review_type'] = 'positive'
request_params['purchase_type'] = 'steam'
request_params['day_range'] = '30'
net_nodes = get_non_ownership_network(app_id, request_params, min_net_size=105)


[appID = 730] expected #reviews = 1073525
Added 76561199657057965 to the network.
Added 76561199682084900 to the network.
Added 76561198836324607 to the network.
Added 76561199278147513 to the network.
Added 76561199675888002 to the network.
Added 76561198245801696 to the network.
Added 76561199186038934 to the network.
Added 76561199198956070 to the network.
Added 76561198289817022 to the network.
Added 76561198302473753 to the network.
Added 76561198824925723 to the network.
Added 76561198961556508 to the network.
Added 76561199239102290 to the network.
Added 76561199524811550 to the network.
Added 76561199560773670 to the network.
Added 76561199642427535 to the network.
Added 76561199671208023 to the network.
Added 76561198148888686 to the network.
Added 76561198148945861 to the network.
Added 76561198180036977 to the network.
Added 76561198253094156 to the network.
Added 76561198283739009 to the network.
Added 76561198351941322 to the network.
Added 76561198359404953 to the network